In [66]:
#2-step keyword extraction
#Text Mining Domains, VU

import pickle
import glob
import pandas as pd
import json
import os

def new_dict_concat_six (unpickled_dict):
    '''
    takes an unpickled dictionary as input with headline as [1] and list of first five sentences as [2]
    converts to a new dict with same key but value is string of concatenated headline + first five sentences
    filters the values of the keywords
    reurns: new dictionary with keys and concatenated headlines and first_five sentences of those articles which 
        contain the keywords
    '''        
    output = [unpickled_dict[key][1]+ '. ' + ' '.join(unpickled_dict[key][2]) for key in unpickled_dict]
    
    # calling test_dict only gets keys
    new_dict = dict(zip(unpickled_dict, output))
    
    return new_dict

def first_filter(keywords, infolder):
    for file in glob.glob(infolder):
        #prep filepaths for writing files
        path = os.path.split(file)[0].split('/')[0]
        basename = os.path.basename(file)

        unpickled_dict = pickle.load(open (file, 'rb'))
        #print(unpickled_dict)
        # run the function above to concat headlines and first_five
        new_dict = new_dict_concat_six(unpickled_dict)
        #print(new_dict)
        #print(new_dict)
        #rough filter for only keywords (see above)
        output_dict = {k: v for k, v in new_dict.items() for keyword in keywords if keyword in v}

        #write filtered articles - choosing json this time so I can inspect them in IDE
        if not os.path.exists(f'filteredpickles/{keywords[0]}'):
            os.makedirs(f'filteredpickles/{keywords[0]}')

        with open(f'filteredpickles/{keywords[0]}/{basename.rstrip(".gz.pkl")}.json', 'w', encoding = 'utf-8') as outfile:
            json.dump(output_dict, outfile)
                  
def keyword_search(keywords, big_df, treshold = 0):
#phase 2 of keyword search
    article_dictlist = []
    #do a keyword search for the company
    i = 0
    for text in big_df['text']:
        keywords_found = set()
        for keyword in keywords:
            if keyword in text:
                keywords_found.add(keyword)
        if len(keywords_found) >= treshold:
            article_dict = {'Identifier' : big_df.index[i], 'Text' : text, 'Keywords' : keywords_found, 'Big_df index' : i}
            article_dictlist.append(article_dict)
        i += 1
    articles = pd.DataFrame(article_dictlist)
    return articles
                  
                  
def no_filter(infolder):
    for file in glob.glob(infolder):
        #prep filepaths for writing files
        path = os.path.split(file)[0].split('/')[0]
        basename = os.path.basename(file)

        unpickled_dict = pickle.load(open (file, 'rb'))
        #print(unpickled_dict)
        # run the function above to concat headlines and first_five
        new_dict = new_dict_concat_six(unpickled_dict)

        #write filtered articles - choosing json this time so I can inspect them in IDE
        if not os.path.exists(f'filteredpickles/unfiltered'):
            os.makedirs(f'filteredpickles/unfiltered')

        with open(f'filteredpickles/unfiltered/{basename.rstrip(".gz.pkl")}.json', 'w', encoding = 'utf-8') as outfile:
            json.dump(new_dict, outfile)

In [60]:
#define the keywords for the 1st rough search
keywords=['poverty', 'Poverty']

#rough filter, choose articles containing 'poverty' or 'aid',
#write them to json files
#computationally the most heavy step

infolder = 'pickles/*'
#create json files for all subcorpora with 1 or more docs containing the keyword(s).
first_filter(keywords, infolder)

In [61]:
keyword = keywords[0]
#read in the filtered json files
df_list = []
for file in glob.glob(f'filteredpickles/{keyword}/*'):
    jsonners = json.load(open(file, 'rb'))
    columns=['text']
    df_list.append(pd.DataFrame.from_dict(jsonners, orient='index', columns= columns))
big_df = pd.concat(df_list)
pd.set_option('display.max_colwidth', None)
print(f'The number of articles left after the first filter is: {len(big_df)}')

The number of articles left after the first filter is: 54455


In [ ]:
no_filter(infolder)

In [70]:
import random


df_list = []
i = 0
while i < 50:
    path = "filteredpickles/unfiltered"
    filename = random.choice(os.listdir(path))
    jsonners = json.load(open(f'{path}/{filename}', 'rb'))
    df_list.append(pd.DataFrame.from_dict(jsonners, orient = 'index', columns = ['text']))
    i+=1

In [87]:
random_df = pd.concat(df_list)
print(len(random_df))

440432


In [62]:
topic = 'poverty'

with open(f'keywords/{topic}.txt', 'r', encoding = 'utf-8') as infile:
    keywords2 = infile.read().splitlines()
    
articles = keyword_search(keywords2, big_df, treshold = 3)
#print(articles)

In [75]:
mixed_df = pd.concat([big_df,random_df])

In [76]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

stopWords = set(stopwords.words('english'))

#to do: figure out how to apply vectorization to the entire corpus
#or: select a representative example to train the tfidfvectorizer!

#2nd to do: find a way to give tf-idf scores to 2-grams and 3-grams

#train and apply Tfidf
vectorizer = TfidfVectorizer(use_idf = True, stop_words = stopWords)
tf_idf_vecs = vectorizer.fit_transform(mixed_df['text'])

In [88]:
#print(tf_idf_vecs)

  (0, 133472)	0.11244104863496775
  (0, 214379)	0.05984875788515843
  (0, 53636)	0.08030536484157844
  (0, 67204)	0.04956203310996382
  (0, 29670)	0.10258812314721022
  (0, 104519)	0.09117467163100387
  (0, 86120)	0.06398747560684362
  (0, 227269)	0.08109966262456396
  (0, 159671)	0.17113556341819738
  (0, 191153)	0.037773790657286574
  (0, 125331)	0.10724811024403348
  (0, 35985)	0.14671437390850908
  (0, 195726)	0.06325560614025016
  (0, 81459)	0.057974556854915496
  (0, 4000)	0.07436434424040826
  (0, 302934)	0.08465238583875216
  (0, 3359)	0.0752879815054523
  (0, 99912)	0.10392272596878264
  (0, 159188)	0.06944762247255404
  (0, 216013)	0.10226390972652007
  (0, 197902)	0.09534031492539272
  (0, 296267)	0.11082363601558565
  (0, 93620)	0.07146050953555214
  (0, 114728)	0.14972901654043722
  (0, 227521)	0.09005140102988617
  :	:
  (494885, 21109)	0.04390941962899101
  (494885, 183020)	0.06155337276798539
  (494885, 140211)	0.053877872929335616
  (494885, 34690)	0.058873156108762544

In [78]:
keywords_corpus = []
for keyword in keywords2:
    if keyword in vectorizer.get_feature_names():
        keywords_corpus.append(keyword)
print('keywords used:', keywords_corpus)
            

keywords used: ['vulnerable', 'poor', 'ownership', 'control', 'property', 'resilience', 'exposure', 'shocks', 'disasters', 'mobilization', 'inhuman', 'workers', 'extortion', 'slavery', 'famine', 'hunger', 'entrepreneur', 'development']


In [79]:
#third step:
#compute tf-idf scores for selected docs. takes a few minutes
doc_scores = []
for i in articles['Big_df index']:
    df = pd.DataFrame(tf_idf_vecs[i].T.todense(), index=vectorizer.get_feature_names(), columns=["tfidf"])
    df = df.sort_values(by=["tfidf"], ascending=False)
    #print(df)
    doc_score = 0
    keywords_found = set()
    for keyword in keywords_corpus:
        keyword_score = df['tfidf'][keyword]
        if keyword_score > 0:
            keywords_found.add(keyword)
            doc_score += keyword_score
    if len(keywords_found) > 0:
        doc_score = doc_score / len(keywords_found)
    doc_scores.append(doc_score)


Empty DataFrame
Columns: []
Index: [00, 000, 0000, 0001, 0001gmt, 000466, 0005, 000696, 000896, 0009, 000m, 000th, 000us, 001, 0010, 0012, 0015, 001748, 002, 0024, 0026, 003, 0030, 0031, 0033, 0035, 0037, 0038, 0039, 004, 0040, 0041, 0044, 0045, 0046, 0047, 0049, 005, 0050, 0051, 0054, 0055, 0056, 0057, 0058, 0059, 006, 0060, 0062, 0063, 0065, 0066, 0069, 00696, 007, 0070, 0073, 0075, 0079, 008, 0081, 0086, 0087, 0089, 0090, 0091, 0097, 009704, 0098, 009905, 009908, 00pm, 00s, 01, 010, 0100, 010004, 010010, 0100gmt, 0101, 011, 0110, 0115, 01181906, 01181911, 0119, 012, 0127, 013, 0130, 0134, 0135, 0136, 0139, 014, 0140, 0141, 015, 0150, 0157, ...]

[107547 rows x 0 columns]


In [42]:
print(df['tfidf'])

fijian         0.656915
waqa           0.257551
dollars        0.255263
fiji           0.236556
labor          0.167536
                 ...   
floodplains    0.000000
floodlit       0.000000
floodlights    0.000000
floodings      0.000000
äcars          0.000000
Name: tfidf, Length: 107547, dtype: float64


In [80]:
scores_df = pd.DataFrame(doc_scores, columns = ['Score'])

In [81]:
articles_w_scores = pd.concat([articles, scores_df], axis = 1)

In [83]:
articles_complete = articles_w_scores.drop('Big_df index', axis=1)

In [84]:
articles_complete = articles_complete.sort_values(by="Score", ascending = False)

In [89]:
articles_complete[:50]

Identifier  \
410   APW_ENG_20041110.0267   
293   APW_ENG_19990227.0166   
903   XIN_ENG_19950427.0232   
813   NYT_ENG_20021110.0186   
1113  XIN_ENG_20000129.0124   
1236  XIN_ENG_20011015.0247   
1173  XIN_ENG_20001219.0309   
1626  XIN_ENG_20080320.0286   
301   APW_ENG_19990624.0855   
153   AFP_ENG_20061202.0007   
1103  XIN_ENG_19991012.0230   
1095  XIN_ENG_19990807.0152   
450   APW_ENG_20060222.0356   
259   APW_ENG_19960920.0621   
261   APW_ENG_19960920.0682   
436   APW_ENG_20050915.1138   
349   APW_ENG_20020626.0840   
887   XIN_ENG_19950207.0020   
372   APW_ENG_20021202.0215   
955   XIN_ENG_19961031.0062   
1303  XIN_ENG_20021016.0102   
348   APW_ENG_20020626.0709   
421   APW_ENG_20050501.0168   
834   NYT_ENG_20050722.0279   
1706  XIN_ENG_20091211.0173   
346   APW_ENG_20020618.1182   
1701  XIN_ENG_20091202.0200   
1064  XIN_ENG_19981208.0100   
430   APW_ENG_20050824.1006   
569   APW_ENG_20100324.0138   
924   XIN_ENG_19951004.0099   
1094  XIN_ENG_19990713.0186   
1098  XIN_ENG_19990910.0019   
1459  XIN_ENG_20050517.0302   
479   APW_ENG_20070217.0290   
514   APW_ENG_20080311.0017   
1080  XIN_ENG_19990316.0304   
426   APW_ENG_20050625.0003   
1294  XIN_ENG_20020904.0243   
516   APW_ENG_20080404.0010   
418   APW_ENG_20050225.0397   
1209  XIN_ENG_20010605.0050   
1292  XIN_ENG_20020902.0249   
203   AFP_ENG_20090112.0779   
161   AFP_ENG_20070810.0520   
1562  XIN_ENG_20070226.0236   
1351  XIN_ENG_20031008.0147   
1695  XIN_ENG_20091014.0134   
211   AFP_ENG_20090420.0571   
1560  XIN_ENG_20070215.0186   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   